In [123]:
import numpy as np
import pandas as pd
from sklearn import metrics

import torch 
import torchvision
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F

In [89]:
df = pd.read_csv("/home/mikey/Datasets/cars.csv")

In [90]:
df.drop(['Car', 'Model'], inplace = True, axis = 1)

In [91]:
y  = df.pop('CO2')


In [92]:
X = df
X

,Volume,Weight
0,1000,790
1,1200,1160
2,1000,929
3,900,865
4,1500,1140
5,1000,929
6,1400,1109
7,1500,1365
8,1500,1112
9,1600,1150


In [93]:
df.head()

,Volume,Weight
0,1000,790
1,1200,1160
2,1000,929
3,900,865
4,1500,1140


In [94]:
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Volume  36 non-null     int64
 1   Weight  36 non-null     int64
dtypes: int64(2)
memory usage: 704.0 bytes


In [95]:
X = X.to_numpy().astype('float32')
y = np.array(y, dtype = 'float32')

In [96]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [97]:
train_dataset = TensorDataset(X, y)

In [98]:
train_dataset = DataLoader(train_dataset, 4, shuffle = True)

In [99]:
for d in train_dataset:
    print(d)
    break

[tensor([[1400., 1109.],
        [1000.,  929.],
        [1600., 1119.],
        [2000., 1705.]]), tensor([ 90.,  95., 104., 114.])]


In [119]:
model = nn.Linear(2, 1)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2147, -0.1878]], requires_grad=True)
Parameter containing:
tensor([-0.2762], requires_grad=True)


In [125]:
preds = model(X)
metrics.mse(y, preds)

AttributeError: module 'sklearn.metrics' has no attribute 'mse'

In [113]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1685, -0.6337]], requires_grad=True),
 Parameter containing:
 tensor([0.4056], requires_grad=True)]

In [117]:
loss_fn = F.mse_loss

loss = loss_fn(model)

In [118]:
opt = torch.optim.SGD(model.parameters(), lr = 0.001)

In [149]:
def fit(num_epochs, model, loss_f, optim):
    for epoch in range(num_epochs):
        
        
        for xb, yb in  train_dataset:
            # 1. Generate preds
            pred = model(xb)

            # 2. Calculate the loss
            loss = loss_fn(pred, yb)
            # compute the grads
            loss.backward()
            # 4. Update the parameters using gradients
            opt.step()
            # 5. Reset the gradients to zero
            opt.zero_grad()
        # print the progress
        if (epoch+1)%10 == 0:
            print('Epoch [{}/{}], loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        

In [150]:
fit(100, model, loss, opt)

<ipython-input-149-9a95a5da6450>:10: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_fn(pred, yb)


Epoch [10/100], loss: 428851.7500
Epoch [20/100], loss: 345478.6250
Epoch [30/100], loss: 465339.3438
Epoch [40/100], loss: 505643.7500
Epoch [50/100], loss: 553717.5625
Epoch [60/100], loss: 589993.6875
Epoch [70/100], loss: 467572.8125
Epoch [80/100], loss: 318237.5312
Epoch [90/100], loss: 522032.2500
Epoch [100/100], loss: 454198.6562
